In [ ]:
import utils
import Simple_CNN

import matplotlib.pyplot as plt

import torch
import torchsummary
from torchvision import transforms

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split

from torch.utils.data import DataLoader

from sklearn.preprocessing import OneHotEncoder


import numpy as np
from importlib import reload

import yaml

In [ ]:
with open("SimpleCNN.yaml", "r") as f:
  net_archi = yaml.load(f, Loader=yaml.FullLoader)
  
  print(net_archi)

In [ ]:
# Parameters
random_seed = 42

In [ ]:
reload(utils)

transform_img = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
    ]
)

transform_labels = transforms.Compose(
    [transforms.ToTensor()]
)

dataset = utils.SpectrogramDataset("../datasets/Aves-Mammalia.csv", 
                                   "../spectrograms/small_dataset", 
                                   "gbifID", "class", 
                                   transform=transform_img, 
                                   one_hot_encode_labels=True)
dataset.show_sample(8, figsize=(20, 10))

In [ ]:
test_percentage = .2
test_len = int((len(dataset) * test_percentage))
train_len = len(dataset) - test_len

train, test = random_split(dataset, [train_len, test_len], torch.Generator().manual_seed(random_seed))

train_loader = DataLoader(train, batch_size=16)
test_loader = DataLoader(test, batch_size=16)

In [ ]:
len(train_loader.dataset), len(test_loader.dataset)

In [ ]:
net_archi["SimpleCNN2out"]

In [ ]:
reload(Simple_CNN)
net = Simple_CNN.SimpleCNN(net_archi["SimpleCNN2out"])

net = utils.cuda_network(net)

criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=.001)

# torchsummary.summary(net, (3, 1024, 256))

In [ ]:
net.parameters

In [ ]:
torchsummary.summary(net, (3, 1024, 256))

In [ ]:
torch.cuda.empty_cache()
((train_loss, train_acc),
 (val_loss, val_acc)) = net.fit(10, train_loader, test_loader, criterion, optimizer,
                             log_file="train1.log", plot_file="train1.png")

In [ ]:
train_loss, train_acc, val_loss, val_acc